In [ ]:
import torch
from torchvision import transforms, models
from PIL import Image
import gradio as gr

torch.manual_seed(42)  # For reproducibility

# 1. Load class names
class_names = ['pizza', 'steak', 'sushi']

# 2. Load your saved model
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = torch.nn.Sequential(
    torch.nn.Linear(model.fc.in_features, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(256, 3)
)
model.load_state_dict(torch.load("best_model_resnet50.pth", map_location="mps"))
model.eval()

# 3. Define image transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# 4. Prediction function
def predict(img):
    img = transform(img).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        outputs = model(img)
        probs = torch.softmax(outputs, dim=1)
    top_prob, top_class = torch.max(probs, dim=1)
    return {class_names[i]: float(probs[0][i]) for i in range(len(class_names))}

# 5. Gradio UI
gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=3),
             title="🍽️ Food Classifier",
             description="Upload an image of steak, pizza or sushi!").launch()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
